# Exploración Inicial de los Datos de RCP Telefónica

Este notebook contiene los primeros pasos para analizar los datos del proyecto de RCP telefónica.

## Importar Librerías Necesarias

Importamos las librerías principales para el análisis de datos y visualización.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de gráficos
%matplotlib inline
sns.set(style="whitegrid")

## Cargar y Visualizar los Datos

Cargamos los archivos de datos desde la carpeta `data` y mostramos las primeras filas de cada uno.

In [ ]:
# Cargar el archivo cleaned_data.csv y mostrar el número de filas
df = pd.read_csv('../data/cleaned_data.csv')
print(f"Número de filas cargadas: {df.shape[0]}")
df.head()

In [4]:
# Cargar los datos
import os

# Listar archivos en la carpeta data
archivos = os.listdir('../data')
print('Archivos en data:', archivos)

# Cargar archivos CSV relevantes
csv_files = [f for f in archivos if f.endswith('.csv')]
dfs = {}
for file in csv_files:
    df = pd.read_csv(f'../data/{file}')
    dfs[file] = df
    print(f'\nPrimeras filas de {file}:')
    display(df.head())

Archivos en data: ['raw_data_cpr.csv', 'Rosario Conde RCP TranstelefÃ³nica.xlsx', 'raw_data_cpr_survive.csv', 'cleaned_data.csv', 'cleaned_data_with_missing.csv']

Primeras filas de raw_data_cpr.csv:


,NUM INFORME;FECHA_LLAMADA;CODIGO_INICIAL;CODIGO FINAL;CODIGO PATOLOGICO;EDAD;SEXO;RCP_TRANSTELEFONICA;FC;FR;TAS;SAT O2;Consulta;Antecedentes;Técnicas;Evolución;Hospital;6 HORAS;24 horas;7 días;C0_C1;C1_C2;C2_C3;C3_C4;C4_C5;C5_FIN
0,1122;03/01/2024 19:03;3.1 - PARADA CARDIORESPI...
1,DIABETES TIPO 2
2,PROTESIS MIIZDO
3,TTO AAS GABAPENTINA ATORVASTATINA INSULINA SE...
4,3937;12/01/2024 08:00;3.5 - INTOXICACIÓN POR D...


ParserError: Error tokenizing data. C error: Expected 5 fields in line 5, saw 7


## Exploración Inicial de los Datos

Revisamos la estructura, tipos de variables y valores nulos de los datos principales.

In [ ]:
# Exploración de estructura y valores nulos
for nombre, df in dfs.items():
    print(f'\nInformación de {nombre}:')
    display(df.info())
    print(f'\nValores nulos en {nombre}:')
    display(df.isnull().sum())

## Estadísticas Descriptivas Básicas

Generamos estadísticas descriptivas para las variables numéricas de los datos principales.

In [ ]:
# Estadísticas descriptivas
for nombre, df in dfs.items():
    print(f'\nEstadísticas descriptivas de {nombre}:')
    display(df.describe())

## Visualización de Distribuciones de Variables

A continuación, se muestran histogramas y diagramas de caja para las variables principales.

In [ ]:
# Visualización de variables numéricas
for nombre, df in dfs.items():
    num_cols = df.select_dtypes(include=[np.number]).columns
    if len(num_cols) > 0:
        print(f'\nHistogramas de {nombre}:')
        df[num_cols].hist(bins=20, figsize=(12, 6))
        plt.suptitle(f'Histogramas de {nombre}')
        plt.show()
        print(f'\nBoxplots de {nombre}:')
        df[num_cols].plot(kind='box', subplots=True, layout=(1, len(num_cols)), figsize=(12, 4), sharey=False)
        plt.suptitle(f'Boxplots de {nombre}')
        plt.show()